In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [3]:
model = "1호모델예측"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'1호모델예측_Y_k3'

In [4]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [5]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [6]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [7]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [8]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [9]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [10]:
drop_togehter = tools.dict_data_drop(list_data.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)
scaled_together["2012-01"].columns

Index(['BW금액', 'CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '유상증자금액', '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액',
       '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
pd.set_option('display.max_columns', 130)

In [12]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 12.59it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 94,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=94)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 71,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=71)},
 '2014-01': {'num_of_cluseer': 3,
  'random_state': 95,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=95)},
 '2015-01': {'num_of_cluseer': 3,
  'random_state': 77,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=77)},
 '2016-01': {'num_of_cluseer': 3,
  'random_state': 8,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=8)},
 '2017-01': {'num_of_cluseer': 3,
  'random_state': 82,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=82)},
 '2018-01': {'num_of_cluseer': 3,
  'random_state': 31,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=31)},
 '2019-01': {'num_of_cluseer': 3,
  'random_state': 18,
  'model': KMeans(max_iter=50

## 모델 기록

In [13]:
size =evaluate.eval_cluster_size(model_save_dict)
#size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

770.0
496.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01,2022-01
0,520.0,497.0,504.0,591.0,635.0,662.0,573.0,581.0,725.0,770.0,726.0
1,504.0,612.0,546.0,553.0,564.0,595.0,595.0,609.0,713.0,691.0,755.0
2,570.0,496.0,570.0,515.0,532.0,530.0,697.0,729.0,539.0,565.0,575.0
기업수,1594.0,1605.0,1620.0,1659.0,1731.0,1787.0,1865.0,1919.0,1977.0,2026.0,2056.0


In [15]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
#eval_one.to_csv("{}_관리종목.csv".format(model_name))
#eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [16]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
#eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
#eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [17]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
#eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
#eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [18]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8919724539761862
1.0


0.10556962025316455

In [19]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

#dist.to_csv("{}_거리분해.csv".format(model_name))
#dist2.to_csv("{}_거리분해평균.csv".format(model_name))

#dist2

예측하는 거 만들기

사용할 기간을 주면 -> 이후 가능한 끝까지의 값을 예측하고, 결과를 리턴하기

In [124]:
# 포착 확률 구하기
key = list(model_save_dict.keys())
key_t1 = key[:-2]
key_t2 = key[1:-1]

return_predict_dict = {}

for t1,t2 in zip(key_t1,key_t2):
    model_t1 = model_save_dict[t1]["model"]
    data = scaled_together[t2]
    #predict_label = model_t1.predict(scaled_together[t2])
    # t2기에 대한 예측값을 담고 있다
    prob, org_prob = Optimizer.eval(model_t1,data,label_data_pp_war[t2])
    return_predict_dict[t2] = prob

return_df = pd.DataFrame()

for t,data in return_predict_dict.items():
    df = pd.DataFrame(data.values(), data.keys(), columns=[str(t)+"기_예측"])
    return_df = pd.concat([return_df,df], axis=1)
return_df

return_df.max(axis=0)#.mean()



return_dict = {}

label_t = eval_two.iloc[-1, :-1]
label_t2 = eval_two.iloc[-1,1:]

for labelt, labelt2, t1,t2 in zip(label_t, label_t2, key_t1,key_t2): 
    mask = model_save_dict[t1]["model"].predict(scaled_together[t2]) == labelt # t기 라벨 사용
    list_corp = scaled_together[t2].loc[mask].index.to_list()

    mask2 = model_save_dict[t2]["model"].labels_ == labelt2 # t+1기 라벨 사용
    list_corp2 = scaled_together[t2].loc[mask2].index.to_list()

    se = set(list_corp+list_corp2)
    return_dict[t2] = len(list_corp)/ len(se)

c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\lib\site-packages\sklearn\ba

In [125]:
return_dict

{'2013-01': 0.9386973180076629,
 '2014-01': 0.9168173598553345,
 '2015-01': 0.9335604770017035,
 '2016-01': 0.9122516556291391,
 '2017-01': 0.9616666666666667,
 '2018-01': 0.9759615384615384,
 '2019-01': 0.998330550918197,
 '2020-01': 0.834733893557423,
 '2021-01': 0.9617346938775511}

In [123]:
np.mean(list(return_dict.values()))

0.9370837948861352

고위험 클러스터의 평균 재무, 공시 상황 출력

* eval_cluster Result의 결과를 받는다 / model_save_dict / raw_data -> drop_together (스케일 전 데이터를 받아야함)

In [21]:
dict = evaluate.see_cluster_info(model_save_dict, eval_one, drop_togehter)

data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)
data.columns = col
data.T.to_csv("{}_평균정보.csv".format(model_name))
data.T

,BW금액,CB금액,경영권분쟁소송,불성실공시법인지정카운트,상호변경카운트,업종변경카운트,유상증자금액,조회공시요구카운트,최대주주변경을수반하는주식담보제공금액,최대주주변경을수반하는주식양수도금액,최대주주변경카운트,타법인주식및출자증권취득결정금액,특수관계인에대한자금대여금액,결산기간_거래량증가,결산기간_주가변동률,당기순이익,매출채권,매출총이익,영업이익,영업현금흐름,차입금의존도,총자본,최대주주비율,판매 및 관리비,최대주주비율_변동성
2012-01,2.400738e+09,4.314455e+08,0.000000,0.128968,0.043651,0.091270,2.300144e+09,0.553571,0.000000e+00,7.507574e+08,0.156746,2.057311e+09,55.555556,inf,1.063940,-3.777913e+06,2.012260e+07,2.271204e+05,-2.328321e+06,-6.611100e+03,28.113601,4.621639e+07,30.523761,2.555445e+06,-0.000252
2012-04,2.471989e+09,5.125237e+08,0.000000,0.116142,0.049213,0.096457,5.085644e+09,0.557087,0.000000e+00,6.879570e+08,0.175197,3.175050e+09,55.118110,inf,1.433503,-6.406811e+06,3.080838e+07,-1.466938e+05,-3.299344e+06,1.023168e+04,28.416772,4.994020e+07,30.114760,3.152650e+06,0.000448
2012-07,3.440211e+09,3.852046e+08,0.000000,0.144050,0.066806,0.031315,4.106494e+09,0.555324,0.000000e+00,7.487205e+08,0.210856,3.570137e+09,363.083507,inf,1.663449,-8.160467e+06,3.514942e+07,-3.729117e+05,-4.019658e+06,1.269067e+05,31.452260,5.185700e+07,28.418412,3.647086e+06,0.004252
2012-10,2.967836e+09,3.258551e+08,0.000000,0.119850,0.071161,0.028090,2.796097e+09,0.464419,0.000000e+00,7.963597e+08,0.177903,3.485396e+09,197.410112,inf,1.552090,-6.983584e+06,3.730828e+07,-2.088975e+05,-3.792168e+06,-6.067115e+05,29.316863,4.984146e+07,30.891906,3.583575e+06,0.005642
2013-01,2.435138e+09,6.012765e+08,0.012072,0.116700,0.078471,0.030181,1.505633e+09,0.466801,0.000000e+00,7.472450e+08,0.191147,2.373943e+09,226.995976,inf,1.555713,-6.421058e+06,3.284179e+07,-6.683174e+05,-3.877266e+06,-1.488904e+06,29.623154,4.368308e+07,30.594343,3.207170e+06,0.005233
2013-04,1.854739e+09,1.558386e+09,0.025743,0.118812,0.079208,0.025743,5.290012e+09,0.419802,0.000000e+00,6.424696e+08,0.205941,1.622975e+09,0.000000,inf,1.796724,-8.893938e+06,5.009446e+07,-8.298119e+05,-4.704903e+06,-2.976989e+06,31.044936,5.255029e+07,31.260458,3.871267e+06,0.007081
2013-07,9.953458e+08,1.366892e+09,0.045187,0.094303,0.074656,0.029470,5.298045e+09,0.392927,0.000000e+00,6.840205e+08,0.188605,1.502599e+09,38.644401,inf,1.577223,-6.655322e+06,4.113471e+07,-2.192109e+05,-3.845842e+06,-2.307262e+06,30.774455,4.286113e+07,31.945560,3.622517e+06,0.008833
2013-10,1.727650e+08,1.493233e+09,0.060194,0.100971,0.066019,0.023301,5.906588e+09,0.366990,0.000000e+00,6.805290e+08,0.203883,1.313258e+09,149.067961,inf,1.526976,-6.738335e+06,4.326271e+07,-2.048564e+05,-3.804366e+06,-2.148501e+06,30.662175,4.720702e+07,31.231692,3.595444e+06,inf
2014-01,1.704664e+08,1.463849e+09,0.067164,0.089552,0.061567,0.022388,4.348308e+09,0.343284,0.000000e+00,4.962880e+08,0.167910,1.309566e+09,0.000000,inf,1.314217,-5.682374e+06,1.734802e+07,-2.827035e+05,-2.634066e+06,-1.090800e+06,28.368218,3.002464e+07,31.893201,2.351362e+06,inf
2014-04,1.884601e+08,1.598660e+09,0.076067,0.074212,0.064935,0.024119,1.785950e+09,0.426716,0.000000e+00,5.804615e+08,0.161410,1.519943e+09,128.701299,inf,1.218040,-2.300942e+06,1.756620e+07,1.476598e+06,-1.047087e+06,-3.675640e+05,28.879165,3.999294e+07,32.366717,2.523685e+06,inf


In [22]:
data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))

In [23]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [24]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,0.781431,-0.053749,0.260742,1,2.5
A000040,-0.283926,0.335411,-0.392740,2,2.5
A000050,0.768327,-0.029412,0.111534,1,2.5
A000070,1.217226,0.279981,0.058730,1,2.5
A000080,1.151922,0.202915,0.157124,1,2.5
...,...,...,...,...,...
A353810,0.054250,0.027571,0.027357,0,2.5
A354200,-0.958266,-0.116695,0.233513,2,2.5
A357780,0.919849,-0.353163,-0.439178,1,2.5
A363280,0.552581,-0.133390,-0.285686,1,2.5
